# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%reload_ext dotenv
%dotenv 

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files_path = glob(os.path.join(PRICE_DATA+"**/*.parquet/*.parquet"))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [11]:
# Write your code below.
import numpy as np

dd_px = dd.read_parquet(parquet_files_path).set_index("ticker")

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low']
)))

C:\Users\Olga\AppData\Local\Temp\ipykernel_14240\2132331097.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [12]:
# Write your code below.

pd_feat = dd_feat.compute()
pd_feat['rolling_avg_return'] = pd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())


In [14]:
pd_feat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2746831 entries, OXY to INTU
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Date                datetime64[ns]
 1   Open                float64       
 2   High                float64       
 3   Low                 float64       
 4   Close               float64       
 5   Adj Close           float64       
 6   Volume              int64         
 7   sector              string        
 8   subsector           string        
 9   year                int32         
 10  Close_lag_1         float64       
 11  returns             float64       
 12  hi_lo_range         float64       
 13  rolling_avg_return  float64       
dtypes: datetime64[ns](1), float64(9), int32(1), int64(1), string(2)
memory usage: 312.4 MB


In [15]:
pd_feat.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,
OXY,2005-01-03,27.956232,27.956232,27.026913,27.089188,16.288534,3756753,Energy,Oil & Gas Exploration & Production,2005,NaN,NaN,0.929319,NaN
OXY,2005-01-04,27.089188,27.175411,26.768236,26.787397,16.107080,4629348,Energy,Oil & Gas Exploration & Production,2005,27.089188,-0.011141,0.407175,NaN
OXY,2005-01-05,25.953884,26.959848,25.949095,26.638899,16.017788,5355607,Energy,Oil & Gas Exploration & Production,2005,26.787397,-0.005544,1.010754,NaN
OXY,2005-01-06,26.638899,27.367023,26.590996,27.146669,16.323101,3806228,Energy,Oil & Gas Exploration & Production,2005,26.638899,0.019061,0.776028,NaN
OXY,2005-01-07,27.146669,27.146669,26.629316,26.705961,16.058109,7627486,Energy,Oil & Gas Exploration & Production,2005,27.146669,-0.016234,0.517353,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No
Yes, Dask is optimized for parallel processing, and converting from Dask to Pandas also requires time and memory.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.